# LLE (Locally Linear Embedding)

LLE e um de varios metodos de reducao de dimensionalidade baseado na teoria spectral. O algoritmo tenta criar uma representacao de forma que os pontos que estao proximos em um espaco de maior dimensionnalidade permanecam proximos na representacao de menor dimensionalidade. 

O LLE cria as representacoes baseando-se apenas nas distancias entre as amostras vizinhas, sem considerar as distancias globais. O algoritmo assume que os dados estejam em um manifold suave (que nao apresente buracos) e que cada ponto esteja posicionados de forma aproximadamente linear aos seus vizinhos. Assumir a ultima parte permite que todos os pontos sejam expressos como uma soma ponderada dos seus vizinhos. A figura abaixo apresenta um manifold que nao e suave.

<img src='assets/holes.jfif' width=400px>

O LLE comeca construindo o mapa conforme os dados originais, que serao replicados nas representacoes com menor dimensao. O algoritmo assume que o dataset seja grande o suficiente e bem distribuido, de modo que tenha pontos suficientes para computar o $k$-NN, e inicia o processo criando a matriz de vizinhanca. Apos definir essa matriz, cada ponto e reconstruido como a soma linear de seus vizinhos. 

O nome LLE (Locally Linear Embedding) se da pela natureza das reconstrucoes. Uma vez que apenas os vizinhos participam da transformacao dos dados, essa reconstrucao e *local*, e e dada pelos coeficientes lineares, ou pesos, e por isso *linear*.

Os pesos contribuem para a reconstrucao dos dados em em uma maior dimensionalidade sao os mesmos usados na representacao em uma menor dimensionalidade. Sendo assim, a funcao de custo deve ser definida de modo a nao depender da localizacao das amostras no espaco original, e ser representada por uma funcao de informacao geometrica definida pela matriz de pesos. Esses pesos sao aprendidos usando decomposicao de matriz, a qual gera uma unica solucao onde os pontos reconstruidos sao independentes. Nesse passo, a informacao geometrica da matriz de pesos e incorporada na estrutura global das representacoes.

Vejamos cada etapa de forma mais detalhada

#### Busca pelos vizinhos

A vizinhanca pode ser criada utilizando a tecnica $k$-vizinhos mais proximos ou a abordagem _$\epsilon$-ball neighborhood_ .

**K-nearest neighbor** - Cada ponto e conectado aos seus $k$-vizinhos mais proximos, o que sempre vai resultar em pelo menos $k$-vizinhos para cada ponto. Note que que cada ponto seleciona exatamente $k$-pontos e pode ser selecionado por algum outro ponto como vizinho, resultando em um numero de vizinhos maior do que o pre-estabelecido. A situacao e comum para casos de pontos isolados que selecionam vizinhos distantes, os quais podem selecionar um conjunto de vizinhos com uma distancia menor. Nesse caso, a matriz e ajustada para ser simetrica.

**$\epsilon$-ball neighbor** - Cada ponto seleciona todos os pontos dentro de uma circunferencia de raio $\epsilon$ e centralizada na sua localizacao como seus vizinhos. As vezes o metodo conduz a pontos sem nenhum vizinho. Alem disso, pode ser dificil encontrar o melhor valor de $\epsilon$ uma vez que valores pequenos podem gerar pontos isolados e valores grandes vao gerar um numero muito grande de vizinhos. Essa abordagem e boa para aproximar distancias geodesicas.

In [1]:
from sklearn.neighbors import NearestNeighbors 
from sklearn import datasets, neighbors
import numpy as np
# vamos implementar o k-nn
# usaremos o dataset iris
iris_ = datasets.load_iris()

iris = iris_.data
def Knbor_Mat(X, K, t = 2.0, dist_metric = "euclidean", algorithm = "ball_tree"):
    
    n,p = X.shape
    
    knn = neighbors.NearestNeighbors(K+1, metric = dist_metric, algorithm=algorithm).fit(X)
    distances, nbors = knn.kneighbors(X)
    
    return(nbors[:,1:])

#### Computando os pesos para reconstrucao

O LLE tambem assume que os dados nao sejam muito ruidosos, de modo que nao existam muitas amostras anomalas. Sendo assim, cada ponto e reconstruido como a media ponderada de seus vizinhos. 

A matriz de pesos tem algumas caracteristicas muito interessante, como por exemplo, e invariante a reescala, rotacao e translacao, e pode ser solucionado utilizando operacoes algebricas em matrizes. 

Uma coisa a se lembrar e que se o numero de vizinhos for maior do que a dimensao original, essa solucao nao sera unica e alguns valores na matriz podem ter valor zero, o que pode ser resolvido adicionando um termo regularizador que penaliza pesos muito grandes.

In [2]:
# calculando os pesos de reconstrucao 
from scipy import linalg

def get_weights(X, nbors, reg, K):
    # n - numero de amostras
    # p - numero de caracteristicas
    n,p = X.shape
    
    # inicializa a matriz de pesos
    Weights = np.zeros((n,n))
    
    # para cada amostra
    for i in range(n):
        # computa a covariancia com cada vizinho
        X_bors = X[nbors[i],:] - X[i]
        cov_nbors = np.dot(X_bors, X_bors.T)
        
        # introduz o termo regularizador
        trace = np.trace(cov_nbors)
        if trace >0 :
            R = reg*trace
        else:
            R = reg
        
        # encontra os pesos dos vizinhos para essa amostra
        cov_nbors.flat[::K+1] += R
        weights = linalg.solve(cov_nbors, np.ones(K).T, sym_pos=True)

        weights = weights/weights.sum()
        Weights[i, nbors[i]] = weights
        
    return(Weights)

#### Computando as representacoes usando a matriz de pesos

Agora seguiremos para o ultimo passo do algoritmo, i.e., computar as representacoes dos dados com a ajuda da matriz de pesos. O erro de reconstrucao e computado utilizando decomposicao de matrizes, e os $p$ menores (com excessao do primeiro, ou seja, do segundo ate $p+1$ menores) autovetores sao usados como nossa nova representacao dos dados.

In [3]:
# computando as representacoes 
from scipy.linalg import eigh

def Y_(Weights,d):
    # n - numero de amostras
    # p - numero de caracteristicas
    n,p = Weights.shape
    # cria uma matriz identidade de lado n
    I = np.eye(n)
    # subtrai os pesos da matriz identidade (todos os valores ficam negativos, com excessao da diagonal principal)
    m = (I-Weights)
    # multiplica a transposta dessa matriz por ela mesmo
    M = m.T.dot(m)
    
    # computa os autovalores e autovetores
    eigvals, eigvecs = eigh(M, eigvals=(1, d), overwrite_a=True)
    ind = np.argsort(np.abs(eigvals))
    
    return(eigvecs[:, ind])
    

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

n_points = 1000
X, color = datasets.samples_generator.make_s_curve(n_points, random_state=0)

def LLE_(X, K):
    reg =0.001
    nbors = Knbor_Mat(X,K)
    print(nbors[0])
    Weights = get_weights(X, nbors, reg,K)
    
    Y = Y_(Weights,2)
    return(Y)
test = [354,520, 246,134,3, 983, 186, 436, 893, 921]
def plotter(K):
    fig = plt.figure(figsize=(10,8))
    Y = LLE_(X , K)
    s = Y[test]
    plt.scatter(Y[:,0],Y[:,1],c=color, cmap=plt.cm.Spectral)
    plt.scatter(s[:,0],s[:,1], c="black")

interact(plotter, K= widgets.IntSlider(min=10, max=100, value=10, step=10))


### Vantagens e desvantagens

LLE tem propriedades interessantes. E um algoritmmo simples que proporciona uma solucao elegante e analitica, sem precisar de iteracoes e computacao de gradientes. A matriz de pesos e esparsa, o que torna a computacao mais facil. Assim como o ISOMAP, uma dificuldade e encontrar o melhor numero de vizinhos, uma vez que esse parametro e muito sensivel e pode proporcionar resultados completamente diferentes. 

# Exercicios

1. Converter o dataset digits usando LLE e comparar a visualizacao das representacoes com as do ISOMAP.